### Imports & Load Data

In [11]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib 

# Load the Vetoed Dataset (The one with NFP/High Spreads set to 0)
df = pd.read_parquet("../data/EURUSD_D1_Ready_for_XGBoost_Vetoed.parquet")

print(f"✅ Loaded {len(df)} rows.")
display(df.tail(3))

✅ Loaded 1485 rows.


,open,high,low,close,tick_volume,spread,real_volume,atr,label,rsi,...,candle_body_rel,sma_50,dist_sma50,atr_rel,day_of_week,month,return_t-1,return_t-2,return_t-3,return_t-5
time,,,,,,,,,,,,,,,,,,,,,
2025-12-11,1.16873,1.17626,1.16822,1.17389,59091,8,0,0.005454,0.0,85.533797,...,0.641791,1.160561,0.011485,1.021191,3,12,0.003857,0.009754,0.008739,0.008176
2025-12-12,1.17364,1.17497,1.17194,1.17390,51017,8,0,0.005324,0.0,85.206718,...,0.085809,1.160557,0.011497,0.998562,4,12,0.000009,0.003865,0.009763,0.008289
2025-12-15,1.17307,1.17688,1.17263,1.17520,48336,8,0,0.005096,0.0,83.302953,...,0.501176,1.160638,0.012547,0.957267,0,12,0.001107,0.001116,0.004977,0.009865


### Data Preparation (The "Clean-Up")

In [12]:
len(df)

1485

In [13]:
df.columns

Index(['open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume',
       'atr', 'label', 'rsi', 'rsi_slope', 'adx', 'plus_di', 'minus_di',
       'trend_strength', 'candle_body_rel', 'sma_50', 'dist_sma50', 'atr_rel',
       'day_of_week', 'month', 'return_t-1', 'return_t-2', 'return_t-3',
       'return_t-5'],
      dtype='object')

In [14]:
# 1. Drop columns the model shouldn't see
# We remove raw prices to force the model to look at PATTERNS (RSI, Distances), not Price Levels.
features_to_drop = [
                    'open', 
                    'high', 
                    'low', 
                    'close', 
                    'tick_volume', 
                    'spread', 
                    'atr', 
                    'label', 
                    'real_volume',
                    # these force it to look at the direction not jsut the raw price
                    'sma_50', 
                    'plus_di',
                    'minus_di'
                    # 'sma_200',
                    # 'atr_rel',
                    # 'bb_width',
                    # 'dist_sma200'
                   ]

X = df.drop(columns=features_to_drop, errors='ignore')
y = df['label']

# 2. Map Labels for XGBoost
# XGBoost requires positive integers (0, 1, 2)
# Our data is -1, 0, 1.
# Map: -1 (Sell) -> 0,  0 (Wait) -> 1,  1 (Buy) -> 2
label_mapping = {-1: 0, 0: 1, 1: 2}
y_mapped = y.map(label_mapping)

# Check
print(f"Features ({len(X.columns)}): {X.columns.tolist()}")
print("\nTarget Class Distribution:")
print(y_mapped.value_counts().sort_index())

Features (13): ['rsi', 'rsi_slope', 'adx', 'trend_strength', 'candle_body_rel', 'dist_sma50', 'atr_rel', 'day_of_week', 'month', 'return_t-1', 'return_t-2', 'return_t-3', 'return_t-5']

Target Class Distribution:
label
0     188
1    1076
2     221
Name: count, dtype: int64


### The Walk-Forward Split

In [15]:
# We split by TIME, not randomly.
# Train: First 80% (The Past)
# Test: Last 20% (The Future relative to training data)

split_point = int(len(df) * 0.80)

X_train = X.iloc[:split_point]
y_train = y_mapped.iloc[:split_point]

X_test = X.iloc[split_point:]
y_test = y_mapped.iloc[split_point:]

print(f"✅ Data Split Successfully:")
print(f"   Train Set: {len(X_train)} days (Past)")
print(f"   Test Set:  {len(X_test)} days (Future - Unseen)")

✅ Data Split Successfully:
   Train Set: 1188 days (Past)
   Test Set:  297 days (Future - Unseen)


### Hyperparameter tuning

In [17]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer, precision_score
import xgboost as xgb
import joblib 

# 1. Define the Custom Scorer
def trading_precision_score(y_true, y_pred):
    precisions = precision_score(y_true, y_pred, average=None, zero_division=0)
    # Average of Sell (0) and Buy (2) precision
    return (precisions[0] + precisions[2]) / 2

custom_scorer = make_scorer(trading_precision_score)

# 2. Define the Parameter Grid
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2, 0.5],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8]
}

# 3. Initialize Base Model (FIX: n_jobs=1)
xgb_base = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=3,
    n_jobs=1,              # <--- CHANGED from -1 to 1 to fix Windows crash
    random_state=42
)

# 4. Time Series Split
tscv = TimeSeriesSplit(n_splits=3)

# 5. Run Randomized Search (FIX: n_jobs=1)
print("🤖 Starting Hyperparameter Search (Single Core Mode)...")
search = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_dist,
    n_iter=100,              # Reduced to 20 to save time since we are on 1 core
    scoring=custom_scorer,  
    cv=tscv,               
    verbose=1,
    random_state=42,
    n_jobs=1                # <--- CHANGED from -1 to 1 to fix Windows crash
)

search.fit(X_train, y_train)

# 6. Show Results
print("\n✅ Best Parameters Found:")
print(search.best_params_)
print(f"Best Validation Precision: {search.best_score_:.4f}")

# 7. Train Final Model
best_model = search.best_estimator_

# Evaluate
y_pred = best_model.predict(X_test)
inverse_map = {0: -1, 1: 0, 2: 1}
y_test_human = y_test.map(inverse_map)
y_pred_human = pd.Series(y_pred).map(inverse_map)

print("\n--- OPTIMIZED MODEL PERFORMANCE ---")
print(classification_report(y_test_human, y_pred_human))

# 8. Save
joblib.dump(best_model, "../models/supervisor_xgb_optimized.joblib")
print("💾 Model Saved.")

🤖 Starting Hyperparameter Search (Single Core Mode)...
Fitting 3 folds for each of 100 candidates, totalling 300 fits

✅ Best Parameters Found:
{'subsample': 0.6, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.03, 'gamma': 0.1, 'colsample_bytree': 0.7}
Best Validation Precision: 0.2456

--- OPTIMIZED MODEL PERFORMANCE ---
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        35
           0       0.73      0.91      0.81       216
           1       0.23      0.11      0.15        46

    accuracy                           0.68       297
   macro avg       0.32      0.34      0.32       297
weighted avg       0.57      0.68      0.61       297

💾 Model Saved.


### The Report Card (Evaluation)

### Save the Supervisor

In [18]:
from sklearn.metrics import precision_score

# 1. Get the Raw Probabilities (Not just the label)
# Shape: [Rows, 3] -> Columns are [Prob_Sell, Prob_Wait, Prob_Buy]
y_probs = best_model.predict_proba(X_test)

# 2. Define a "Strictness" Test
thresholds = [0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70]

print(f"{'Threshold':<10} | {'Trades':<8} | {'Precision (Buy)':<15} | {'Precision (Sell)':<15}")
print("-" * 60)

for t in thresholds:
    # Logic: Only pick a side if Probability > Threshold
    # We create a new list of predictions based on strictness
    
    custom_preds = []
    
    for probs in y_probs:
        p_sell, p_wait, p_buy = probs
        
        if p_buy > t:
            custom_preds.append(2)  # High confidence Buy
        elif p_sell > t:
            custom_preds.append(0)  # High confidence Sell
        else:
            custom_preds.append(1)  # Not sure? Wait.
            
    # Calculate Precision for this threshold
    # Note: We are comparing 'custom_preds' vs 'y_test_human' mapped to 0,1,2
    # Map y_test to 0,1,2 for calculation
    y_true_mapped = y_test # Already 0,1,2
    
    prec = precision_score(y_true_mapped, custom_preds, average=None, zero_division=0)
    
    # Count how many trades we actually took (Count of 0s and 2s)
    trade_count = custom_preds.count(0) + custom_preds.count(2)
    
    print(f"{t:<10} | {trade_count:<8} | {prec[2]:.4f}          | {prec[0]:.4f}")

Threshold  | Trades   | Precision (Buy) | Precision (Sell)
------------------------------------------------------------
0.4        | 28       | 0.2381          | 0.0000
0.45       | 15       | 0.2857          | 0.0000
0.5        | 5        | 0.4000          | 0.0000
0.55       | 2        | 0.5000          | 0.0000
0.6        | 1        | 0.0000          | 0.0000
0.65       | 0        | 0.0000          | 0.0000
0.7        | 0        | 0.0000          | 0.0000


In [ ]:
model_filename = "../models/supervisor_xgb.joblib"
joblib.dump(model, model_filename)
print(f"💾 Model successfully saved to {model_filename}")